In [ ]:
// Using
#r "../NxtStp.Net.LiveLogger-main/NxtStpHttpClientLib/bin/Debug/net472/NxtStpHttpClientLib.dll"
using NxtStpHttpClientLib;
using System.Text.Json;
using System.Diagnostics;
using System.Threading;
using System.Net;

// shared classes
public class NotebookSpecficObject
{
    private int itemIndex;

    public int ItemIndex
    {
        get => itemIndex;

        set => itemIndex = value;
    }

    private int delay;

    public int Delay
    {
        get => delay;
        set => this.delay = value;
    }

    private int threadId;

    public int ThreadId
    {
        get => this.threadId;

        set => this.threadId = value;
    }
}

internal class NxtStpHttpPerformanceNotebookLog : NxtStpHttpPerformanceLog
{
    public override void BeginLog(params object[] objects)
    {
        base.BeginLog(objects); // log starting time, and store it to M0
        this.MessagesDictionary.Add("ItenIndex", ((NotebookSpecficObject)objects[0]).ItemIndex.ToString());
        this.MessagesDictionary.Add("Delay", ((NotebookSpecficObject)objects[0]).Delay.ToString());
        this.MessagesDictionary.Add("Begin thread Id", ((NotebookSpecficObject)objects[0]).ThreadId.ToString());
    }

    public override void EndLog(params object[] objects)
    {
        this.MessagesDictionary.Add("End thread Id", ((int)objects[0]).ToString());
        base.EndLog(objects);
    }
}

# LiveLogger
#### Log information on a live dashboard from within an application as you use it.
<b>What</b>
<ul>
<li>measure execution speed.</li>
<li>follow execution order.</li>
<li>log exception message.</li>
</ul>
<b>Where</b>
<ul>
<li>Work with distributed application too.</li>
</ul>
<b>When</b>
<ul>
<li>When IDE integrated debugging tools are not enough.</li>
</ul>

In [ ]:
sequenceDiagram
    Participant app as Application
    box grey DotNet http client 
    participant http as NxtStpHttpClient
    end
    box grey Node server
    participant node as NxtStp.Node.LiveLogger.Main.js
    end
    box grey Simple logger as html page <br>with embeded javascript and css <br> served directly by <br>the node server.
    participant simplepage as index.html
    end
    box grey Angular logger.
    participant angularlogger as NxtStplivelogger-Frontend
    end
    app ->> http : EndPointPost(json string payload)
    http ->> node : log <br>End point
    node ->> simplepage : websocket
    simplepage ->> node : 
    node ->> angularlogger : websocket
    angularlogger ->> node : 

### Log information encoded as json object using the log end point
One key, one value<br>
In the angular application, this create a table with one column and one row

In [ ]:
//Example 1
var http = new NxtStpHttpClient();
public class JsonPayLoad{
    public string Label {get; set;}
    public string Value1 {get; set;}
    public string Value2 {get; set;}
}
var payloadObject = new  JsonPayLoad(){
    Label = "Test2",
    Value1 = "1.0",
    Value2 = "1.0"
};
var options = new JsonSerializerOptions { WriteIndented = true };
string jsonPayLoad = JsonSerializer.Serialize(payloadObject, options);
http.EndPointPost(jsonPayLoad);
Console.WriteLine(jsonPayLoad);

{
  "Label": "Test2",
  "Value1": "1.0",
  "Value2": "1.0"
}


### Log information encoded as plain text using the logText end point
In the angular application, this does add an item in the *message of type plain text* section of the accordeon item *Unprocessed meassages*

In [ ]:
// Example 2
var http = new NxtStpHttpClient("127.0.0.1", "logText", "text/plain");
var textPayLoad = "Hello world!";
http.EndPointPost(textPayLoad);
Console.WriteLine(textPayLoad);

### Log execution duration in single thread context and for a single path

In [ ]:
classDiagram
    class INxtStpHttpClient{
    <<interface>>
    get; Id : Guid 
    get; set; Metrics : NxtStpHttpClientMetrics
    EndPointPost(string) void}

    class NxtStpHttpClient
    note for NxtStpHttpClient "low level API for posting log"

    INxtStpHttpClient <|.. NxtStpHttpClient

    class NxtStpHttpClientMetrics{
    +M0: long
    +M1: long
    +M3: long
    +M4: long
    +PerformanceStopWatchElapsed() long}
    class StopWatch
    StopWatch ..> NxtStpHttpClientMetrics : constructor injection
    NxtStpHttpClientMetrics ..> NxtStpHttpClient : setter injection
    note for NxtStpHttpClientMetrics "Provide functionality for measuring execution time"

    class NxtStpHttpPerformanceLogBase{
    -memExecutionPath: Dictionary~int, string~
    +MessagesDictionary : 
    +M0: string
    +M1: string
    +M2: string
    +M3: string
    +BeginLog(pathIndex : int)
    +EndLog(pathIndex : int, description : string)
    +BeginLog(~params~ pathIndexes : int[])
    +EndLog(~params~ pathIndexes : int[]
    
    +DictToJsonString(...) string
    +PerformanceStopWatchElapsed() string
    }
    NxtStpHttpClient <|-- NxtStpHttpPerformanceLogBase
    note for NxtStpHttpPerformanceLogBase "Higher level base class API.
    path is an idetifier for an execution path.

    <b>memExecutionPath</b> is a private dictonary for which 
    the key is the path index and the string 
    are elapsed time in text form 
    when BeginLog is called. This dictionary is
    used to build
    
    <b>NessagesDictionary</b>. This dictonary
    contains path as keys and path execution duration as value.

    BeginLog(&lt;params&gt; pathIndexes : int[]) is used for paths 
    that does start simultaneously. 
    
    EndLog(&lt;params&gt; pathIndexes : int[]) is used to indicate 
    the execution paths are not executed"

In [ ]:
// Example 3
// performance measurement using low level API
var stopWatch = new Stopwatch();
stopWatch.Start();
var http = new NxtStpHttpClient("127.0.0.1", "logText", "text/plain") { Cloud = false};
http.Metrics = new NxtStpHttpClientMetrics(stopWatch);
var metrics = http.Metrics;
metrics.M0 = http.Metrics.PerformanceStopWatchElapsed();
// a long runing task
Thread.Sleep(2024);
// end
metrics.M1 = http.Metrics.PerformanceStopWatchElapsed();
var metric1 = metrics.MicroSecondsToSeconds(metrics.M1-metrics.M0);
Console.WriteLine(metrics.MicroSecondsToSeconds(metrics.M1-metrics.M0));
var log =$"Metric low level API: {metric1}";
Console.WriteLine(log);
http.EndPointPost(log);


In [ ]:
// Example 4
// performance measurement  using minimum functionalities of the NxtStpHttpPerformanceLogBase base class.
var httpPerformanceBase = new NxtStpHttpPerformanceLogBase();
httpPerformanceBase.BeginLog(1);
// a long runing task
Thread.Sleep(2024);
// end
httpPerformanceBase.EndLog(1, "The execution path description");
var dict = httpPerformanceBase.MessagesDictionary;
var log =  httpPerformanceBase.DictToJsonString(dict);
Console.WriteLine(log); // the dictionary contains two kind keys:
// - Execution path # 
// - Path execution duration [s] #. 
// These two set of keys translates into two columns. The first column provide a description of the path 1 and the second column provide the execution duration of this path.
var http = new NxtStpHttpClient();
http.EndPointPost(log);


### Log execution duration of multiple paths

In [ ]:
flowchart
    A[Start] --> B{Path 1, 2 or 3?}
    B -->|path 1|E[End]
    B -->|path 2|E[End]
    B -->|path 3|E[End]

In [ ]:
// example 5
// rather verbose mutipath logging

var httpPerformanceBase = new NxtStpHttpPerformanceLogBase();
enum Paths {
    path1,
    path2,
    path3
}
var path = Paths.path2;
httpPerformanceBase.BeginLog(1, 2, 3); // start time for path 1, 2 and 3 are same and measured at this point.
switch (path) { // one path is executed
    case Paths.path1:
        Thread.Sleep(1000);
        httpPerformanceBase.EndLog(1, "Path 1 executed");
        httpPerformanceBase.EndLog(2, 3); // set path 2 and 3 as not executed
        break;
    case Paths.path2:
        Thread.Sleep(2000);
        httpPerformanceBase.EndLog(2, "Path 2 executed");
        httpPerformanceBase.EndLog(1, 3); // set path 1 and 3 as not executed
        break;
    case Paths.path3:
        Thread.Sleep(3000);
        httpPerformanceBase.EndLog(3, "Path 3 executed");
        httpPerformanceBase.EndLog(1, 2); // set path 1 and 2 as not executed
        break;
}
httpPerformanceBase.EndLog();
var dict = httpPerformanceBase.MessagesDictionary;
var log =  httpPerformanceBase.DictToJsonString(dict);
Console.WriteLine(log); // the dictionary contains two kind keys:
// - Execution path # 
// - Path execution duration [s] #. 
// These two set of keys translates into two columns. The first column provide a description of the path 1 and the second column provide the execution duration of this path.
var http = new NxtStpHttpClient();
http.EndPointPost(log);

### Log in a multithread environement
**working principle**
- for each thread a http client is created that maintains states like time information (log begin time and end time)
- http instance are no recreated but reuse whenever possible. There is one http in stance per thread id.
- log information are send only at the end as batch to not affect execution time of the method under investigation

In [ ]:
classDiagram
    class NxtStpHttpClientsContainer~TInstanceIndexType~{
    -httpsContainer : ConcurrentDictionary~TInstanceIndexType, INxtStpHttpClient~
    +GetHttp~TLogType~(id : TInstanceType) TLogType
    }
    class INxtStpHttpClient{
    <<interface>>
    get; Id : Guid 
    get; set; Metrics : NxtStpHttpClientMetrics
    EndPointPost(string) void}
    note for INxtStpHttpClient "NxtStpHttpClientMetrics encapsulate <br>performance measurement functionality."
    note for NxtStpHttpClientsContainer "Manage a dictionary of http clients, 
    one per thread.
    TInstanceType can be int 
    or Guid for example."
    class TLogType
    INxtStpHttpClient <|.. TLogType
    TLogType <.. NxtStpHttpClientsContainer : GetHttp return TLogType
    note for TLogType "Is a class typically derived 
    from NxtStpHttpPerformanceLogBase.
    The derived class takes typically 
    host object as parameter that 
    provide information to log."

    class TInstanceIndexType
    note for TInstanceIndexType "Is typically a simple type 
    like int (same type as 
    managed thread Id)"

    class NxtStpHttpClientFactory~TInstanceIndexType~{
     Lazy(ref NxtStpHttpClientsContainer~TInstanceIndexType~ nxtStepHttpClientsContainer) NxtStpHttpClientsContainer~TInstanceIndexType~ 
    }
    note for NxtStpHttpClientFactory "Lazy method return either 
    a new instance of 
    NxtStpHttpClientsContainer 
    or the existing one.
    
    There is an overload Lazy method 
    that also log how long to create 
    the container and the specific 
    http client associated 
    to each thread context."

    TInstanceIndexType <.. NxtStpHttpClientFactory : use
    TInstanceIndexType <.. NxtStpHttpClientsContainer : use as key in httpsContainer
    

In [ ]:
// Example 6
// Example that illustrate the use of HttpClientFactory to create an instance of NxtStpHttpClientsContainer and log the time it take to create those instances

NxtStpHttpClientsContainer<int> httpClientsContainer; 
Console.WriteLine($"Thread # {Thread.CurrentThread.ManagedThreadId}");
var nxtStpHttpPerformanceLogBase = NxtStpHttpClientFactory<int>.Lazy(ref httpClientsContainer, IPAddress.Parse("127.0.0.1"), false, "Polyglot notebook context", "context not use").GetHttp<NxtStpHttpPerformanceLogBase>(Thread.CurrentThread.ManagedThreadId);
nxtStpHttpPerformanceLogBase.BeginLog(1);
Thread.Sleep(1000);
nxtStpHttpPerformanceLogBase.EndLog(1, "path 1");
var dict = nxtStpHttpPerformanceLogBase.MessagesDictionary;
var log =  nxtStpHttpPerformanceLogBase.DictToJsonString(dict);
Console.WriteLine(log); // the dictionary contains two kind keys:
// - Execution path # 
// - Path execution duration [s] #. 
// These two set of keys translates into two columns. The first column provide a description of the path 1 and the second column provide the execution duration of this path.
var http = new NxtStpHttpClient();

// log information about execution path 1
http.EndPointPost(log);

//Log metrics about container and nxtStpHttpPerformanceLogBase creation.
var metricsDictionary = nxtStpHttpPerformanceLogBase.Metrics.MetricsDictionary;
log =  nxtStpHttpPerformanceLogBase.DictToJsonString(metricsDictionary);
Console.WriteLine(log);
http.EndPointPost(log);



In [ ]:
// Example 7

// Example that illustrate the instantiation of a class derived from NxtStpHttpPerformanceLogBase using the a factoring. This class encapsulate a NxtStpHttpClient and 
// also takes an object that is specific to the host as paramter in the BeginLog method so that information of thtat class can be logged.

public class NotebookSpecficObject 
{
    public string Info => "Notebook specific information";
}

var notebook = new NotebookSpecficObject();

public class NxtStpHttpPerformanceNotebookLog: NxtStpHttpPerformanceLog
{
    /// <summary>
    /// 
    /// </summary>
    public override void BeginLog(params object[] objects)
    {
        base.BeginLog(objects); // log starting time, and store it to M0
        this.MessagesDictionary.Add("Notebook", ((NotebookSpecficObject)objects[0]).Info);
    }
}

NxtStpHttpClientsContainer<int> httpClientsContainer; 
Console.WriteLine($"Thread # {Thread.CurrentThread.ManagedThreadId}");
var nxtStpHttpPerformanceNotebookLog = NxtStpHttpClientFactory<int>.Lazy(ref httpClientsContainer).GetHttp<NxtStpHttpPerformanceNotebookLog>(Thread.CurrentThread.ManagedThreadId);

nxtStpHttpPerformanceNotebookLog.BeginLog(notebook);
Thread.Sleep(1000);
nxtStpHttpPerformanceNotebookLog.EndLog();

In [ ]:
// Example 8
// exemple of tasks run in parallele for which we would like to measure for each the execution time 

using System;
using System.Linq;
using System.Threading;
using System.Threading.Tasks;

   const int workItemNumber = 100;
   const int maximunDurationOfWorkItemExecution = 100; //milliseconds

    public static async Task CreateAndExecuteThreads()
    {
        // Create a list of 10 work items
        var workItemIndexes = Enumerable.Range(1, workItemNumber).ToList();

        // Use Parallel.ForEach to create and run tasks
        await Task.WhenAll(workItemIndexes.Select(itemIndex => PerformTaskAsync(itemIndex)));
    }

    private static async Task PerformTaskAsync(int itemIndex)
    {
        // Create a random number generator
        Random random = new Random();

        // Generate a random number of milliseconds (less than 1000 ms)
        int delay = random.Next(maximunDurationOfWorkItemExecution);

        // Wait for the random number of milliseconds
        await Task.Delay(delay);

        // Log the item, delay, and thread ID
        Console.WriteLine($"Item index: {itemIndex}, Delay: {delay} ms, Thread ID: {Thread.CurrentThread.ManagedThreadId}");
    }

    await CreateAndExecuteThreads();



In [ ]:
// Example 9

var example_9 = new Example_9();
Stopwatch stopwatch = new Stopwatch();
stopwatch.Start();
await example_9.RunAsync(); // execute 100 tasks in a parallele for each look that 
                            // take up to 100 ms seconds each. 
stopwatch.Stop();
TimeSpan ts = stopwatch.Elapsed;
LogEnlapsedTimeInConsole(ts);

private static void LogEnlapsedTimeInConsole(TimeSpan ts)
{
    Console.WriteLine("Elapsed Time: {0:00}:{1:00}:{2:00}.{3:00}", ts.Hours, ts.Minutes, ts.Seconds, ts.Milliseconds / 10);
}

public class Example_9
{
    private const int workItemNumber = 100;

    private const int maximunDurationOfWorkItemExecution = 100; //milliseconds

    private static readonly object LockHttpFactory = new object();

    private static readonly object ConsoleLock = new object();

    private NotebookSpecficObject notebook = new NotebookSpecficObject();

    private NxtStpHttpClientsContainer<int> httpClientsContainer = null;

    internal async Task RunAsync()
    {
        // Example 9
        // exemple of tasks run in parallele for which we would like to measure for each the execution time   

        await CreateAndExecuteThreads();
    }

    private async Task CreateAndExecuteThreads()
    {
        // Create a list of 10 work items
        var workItemIndexes = Enumerable.Range(1, workItemNumber).ToList();

        // Use Parallel.ForEach to create and run tasks
        await Task.WhenAll(workItemIndexes.Select(itemIndex => PerformTaskAsync(itemIndex)));
    }

    private async Task PerformTaskAsync(int workItemIndex)
    {
        int delay;
        lock (LockHttpFactory)
        {
            // Create a random number generator
            Random random = new Random();
            // Generate a random number of milliseconds (less than 1000 ms) 
            delay = random.Next(maximunDurationOfWorkItemExecution);
            notebook.ItemIndex = workItemIndex;
            notebook.Delay = delay;
            notebook.ThreadId = Thread.CurrentThread.ManagedThreadId;
            NxtStpHttpClientFactory<int>.Lazy(ref httpClientsContainer).GetHttp<NxtStpHttpPerformanceNotebookLog>(workItemIndex).BeginLog(notebook);
        }

        // Wait for the random number of milliseconds
        await Task.Delay(delay);
            
        lock (LockHttpFactory)
        {
            NxtStpHttpClientFactory<int>.Lazy(ref httpClientsContainer).GetHttp<NxtStpHttpPerformanceNotebookLog>(workItemIndex).EndLog(Thread.CurrentThread.ManagedThreadId);
        }

        // Log the item, delay, and thread ID
        lock (ConsoleLock)
        {
            Console.WriteLine($"Work item index: {workItemIndex}, Delay: {delay} ms, Thread ID: {Thread.CurrentThread.ManagedThreadId}");
        }
    }

}


Work item index: 79, Delay: 5 ms, Thread ID: 25
Work item index: 16, Delay: 7 ms, Thread ID: 21
Work item index: 43, Delay: 6 ms, Thread ID: 25
Work item index: 41, Delay: 8 ms, Thread ID: 21
Work item index: 11, Delay: 4 ms, Thread ID: 25
Work item index: 1, Delay: 4 ms, Thread ID: 34
Work item index: 59, Delay: 7 ms, Thread ID: 26
Work item index: 8, Delay: 30 ms, Thread ID: 25
Work item index: 49, Delay: 5 ms, Thread ID: 5
Work item index: 9, Delay: 82 ms, Thread ID: 34
Work item index: 13, Delay: 55 ms, Thread ID: 34
Work item index: 14, Delay: 56 ms, Thread ID: 34
Work item index: 15, Delay: 57 ms, Thread ID: 34
Work item index: 17, Delay: 75 ms, Thread ID: 34
Work item index: 10, Delay: 82 ms, Thread ID: 5
Work item index: 19, Delay: 35 ms, Thread ID: 34
Work item index: 20, Delay: 15 ms, Thread ID: 5
Work item index: 21, Delay: 15 ms, Thread ID: 34
Work item index: 22, Delay: 45 ms, Thread ID: 5
Work item index: 24, Delay: 87 ms, Thread ID: 5
Work item index: 25, Delay: 52 ms, T

In [ ]:
// Example 10

Stopwatch stopwatch2 = new Stopwatch();
var example_10 = new Example_10();
stopwatch2.Start();
example_10.Run();
stopwatch2.Stop();
TimeSpan ts2 = stopwatch2.Elapsed;
LogEnlapsedTimeInConsole(ts2);

private static void LogEnlapsedTimeInConsole(TimeSpan ts)
{
    Console.WriteLine("Elapsed Time: {0:00}:{1:00}:{2:00}.{3:00}", ts.Hours, ts.Minutes, ts.Seconds, ts.Milliseconds / 10);
}

private class Example_10
{
    private const int workItemNumber = 100;

    private const int maximunDurationOfWorkItemExecution = 100; //milliseconds

    private static readonly object LockHttpFactory = new object();

    private static readonly object ConsoleLock = new object();

    private NotebookSpecficObject notebook = new NotebookSpecficObject();

    private NxtStpHttpClientsContainer<int> httpClientsContainer = null;

    internal void Run()
    {
        var workItemIndexes = Enumerable.Range(1, workItemNumber).ToList();
        Parallel.ForEach(workItemIndexes, workItemIndex =>
        {
            ExecuteMethod(workItemIndex);
        });
    }

    void ExecuteMethod(int workItemIndex)
    {
        int delay;
        lock (LockHttpFactory)
        {
            // Create a random number generator
            Random random = new Random();
            // Generate a random number of milliseconds (less than 1000 ms) 
            delay = random.Next(maximunDurationOfWorkItemExecution);
            notebook.ItemIndex = workItemIndex;
            notebook.Delay = delay;
            notebook.ThreadId = Thread.CurrentThread.ManagedThreadId;
            NxtStpHttpClientFactory<int>.Lazy(ref httpClientsContainer).GetHttp<NxtStpHttpPerformanceNotebookLog>(workItemIndex).BeginLog(notebook);
        }

        Thread.Sleep(delay);

        lock (LockHttpFactory)
        {
            NxtStpHttpClientFactory<int>.Lazy(ref httpClientsContainer).GetHttp<NxtStpHttpPerformanceNotebookLog>(workItemIndex).EndLog(Thread.CurrentThread.ManagedThreadId);
        }

        lock (ConsoleLock)
        {
            Console.WriteLine($"Work item index: {workItemIndex}, Delay: {delay} ms, Thread Id: {Thread.CurrentThread.ManagedThreadId}");
        }
    }
}

